In [10]:
%matplotlib inline
import os
import sys
import numpy as np
import pandas as np
import seaborn as sb
import xarray as xr
import os.path as path
from aux.ml_flood_config import path_to_data

In [11]:
# define some vars
data_path = f'{path_to_data}danube/'
print(data_path)



/home/srvx11/lehre/users/a1303583/ipython/ml_flood/data/danube/


## rename files if necessary

In [15]:
from aux.rename_files import rename_files

#rename_files(path=data_path, old='day.', new='dayavg.', str_constraint='temperature')

## convert hourly files to daily means

In [16]:
from aux.cdo_daily_means import cdo_daily_means

#cdo_daily_means(path=data_path, file_includes='single_level')
#cdo_daily_means(path=data_path, file_includes='850_700_500')

## convert precipitation values to daily sums

In [17]:
from aux.cdo_precip_sums import cdo_precip_sums

#cdo_precip_sums(path=data_path)